In [1]:
import numpy as np
import os.path as op
arr_dir = "../../Data/3D_map/Map_array"
total_maz = np.load(op.join(arr_dir, 'maz.npy'))
print ('total_maz', total_maz.shape)

total_maz (120, 20, 20, 20)


In [2]:
train_path = np.load(op.join(arr_dir, "train_path.npy"))


print ('train_path', train_path.shape)

train_path (80000, 15, 3)


In [3]:
train_path = np.reshape(train_path, (100, -1, train_path.shape[1], train_path.shape[2]))

print ('train_path', train_path.shape, np.max(train_path), np.min(train_path))


# #select the paths with length less than 12
# all_map = []
# for i in range(train_path.shape[0]):
#     per_map = []
#     for j in range(train_path.shape[1]):
#         if train_path[i,j,-5,0] == train_path[i,j,-1,0] and train_path[i,j,-5,1] == train_path[i,j,-1,1]:
#             per_map.append(train_path[i,j,:-4,:])
#     print ('per_map', len(per_map))
#     all_map.append(per_map[:1000])
# all_map = np.array(all_map)
# train_path = all_map
# print ('new train_path', train_path.shape)

            

train_path (100, 800, 15, 3) 19 0


In [3]:
def augmentation(img, start_x, start_y, start_z, channel=1):
#     start_x, start_y = np.where(img[1]==255)
    h, w = 3, 3
#     for i range(-3, 4):
#         for j in range(-3, 4):
#             x, y = start_x + i, start_y + j
#             if img[0,x,y] != 255:
#                 img[1,x,y] = 255
#     start_x, start_y = np.where(img[2]==255)
    for i in range(-1, 2):
        for j in range(-1, 2):
            for z in range(-1, 2):
                x, y, z = start_x + i, start_y + j, start_z + z
                if 0<=x<=(img.shape[-1]-1) and 0 <=y<=(img.shape[-1]-1) and 0 <=z<=(img.shape[-1]-1) and img[0,x,y,z] != 255:
                    img[channel,x,y,z] = 255
    return img
            

In [4]:
def generate_video(maz, path, seen=True, video_number=0):

    if not video_number:
        video_number = path.shape[1]
    for maz_id in range(path.shape[0]):
        video_per_maz = []
#         for task_id in range(path.shape[1]):
        for task_id in range(video_number):
            start, goal = path[maz_id, task_id, 0], path[maz_id, task_id, -1]
            maz_3channel = np.zeros((3, maz.shape[1], maz.shape[2], maz.shape[3]))
            if seen:
                maz_3channel[0] = np.copy(total_maz[maz_id])
            else:
                maz_3channel[0] = np.copy(total_maz[maz_id+100])
#             maz_3channel[1, start[0], start[1]] = 255
            maz_3channel[2, goal[0], goal[1], goal[2]] = 255
            maz_3channel = augmentation(maz_3channel, goal[0], goal[1], goal[2], 2)
            repeated_maz = np.tile(maz_3channel, (path.shape[2], 1, 1, 1))
            
#             print ('repeated_maz', repeated_maz.shape)
            repeated_maz = np.reshape(repeated_maz, (path.shape[2], 3, maz.shape[1], maz.shape[2], maz.shape[3]))
#             print ('repeated_maz', repeated_maz.shape)
            for path_id in range(0, path.shape[2]):#-1):
                pos = path[maz_id, task_id, path_id]
                repeated_maz[path_id, 1, pos[0], pos[1], pos[2]] = 255
                repeated_maz[path_id] = augmentation(repeated_maz[path_id], pos[0], pos[1], pos[2], 1)
#                 repeated_maz = augmentation(repeated_maz, goal[0], goal[1])
                
#             repeated_maz[path_id+1, 1, pos[0], pos[1]] = 255
#             repeated_maz[path_id+1] = augmentation(repeated_maz[path_id+1], pos[0], pos[1], 1)
            
            
            video_per_maz.append(repeated_maz)
        video_per_maz = np.array(video_per_maz)
#         print ('video_per_maz', video_per_maz.shape)
        video_per_maz = np.expand_dims(video_per_maz, axis = 0)
        if maz_id == 0:
            videos = video_per_maz
        else:
            videos = np.concatenate((videos, video_per_maz), axis=0)
        
#         print ('video_per_maz', video_per_maz.shape)
        print ('videos', videos.shape)
#         videos.append(video_per_maz)
#     videos = np.stack(videos)
    
    return videos
    
                

In [10]:
num = 800
# train_videos = generate_video(total_maz, train_path[50:], True, num)
train_videos = generate_video(total_maz, train_path, True, num)
video_num = train_videos.shape[0] * num
video_len = train_videos.shape[2]

videos (1, 800, 15, 3, 20, 20, 20)
videos (2, 800, 15, 3, 20, 20, 20)
videos (3, 800, 15, 3, 20, 20, 20)
videos (4, 800, 15, 3, 20, 20, 20)
videos (5, 800, 15, 3, 20, 20, 20)
videos (6, 800, 15, 3, 20, 20, 20)
videos (7, 800, 15, 3, 20, 20, 20)
videos (8, 800, 15, 3, 20, 20, 20)
videos (9, 800, 15, 3, 20, 20, 20)
videos (10, 800, 15, 3, 20, 20, 20)
videos (11, 800, 15, 3, 20, 20, 20)
videos (12, 800, 15, 3, 20, 20, 20)
videos (13, 800, 15, 3, 20, 20, 20)
videos (14, 800, 15, 3, 20, 20, 20)
videos (15, 800, 15, 3, 20, 20, 20)
videos (16, 800, 15, 3, 20, 20, 20)
videos (17, 800, 15, 3, 20, 20, 20)
videos (18, 800, 15, 3, 20, 20, 20)
videos (19, 800, 15, 3, 20, 20, 20)
videos (20, 800, 15, 3, 20, 20, 20)
videos (21, 800, 15, 3, 20, 20, 20)
videos (22, 800, 15, 3, 20, 20, 20)
videos (23, 800, 15, 3, 20, 20, 20)
videos (24, 800, 15, 3, 20, 20, 20)
videos (25, 800, 15, 3, 20, 20, 20)
videos (26, 800, 15, 3, 20, 20, 20)
videos (27, 800, 15, 3, 20, 20, 20)
videos (28, 800, 15, 3, 20, 20, 20)
v

In [11]:
train_videos = train_videos/255.0
np.save(op.join(arr_dir, "train_video.npy"), train_videos)
print ('train_video00', train_videos.shape)
print ('max min', train_videos.max(), train_videos.min())

train_video00 (50, 800, 15, 3, 20, 20, 20)
max min 1.0 0.0


In [12]:
train_videos = np.reshape(train_videos, (-1, 3, train_videos.shape[-3], train_videos.shape[-2], train_videos.shape[-1])).astype(np.float32)
print ('train_video', train_videos.shape)

dims = np.array([3, train_videos.shape[-3], train_videos.shape[-2], train_videos.shape[-1]])
dims = np.expand_dims(dims, axis=0)
print (dims)
input_seq = 7
input_start = np.arange(0, video_num*video_len, video_len).astype(np.int32)
output_start = np.arange(input_seq, video_num*video_len, video_len).astype(np.int32)
# input_start = np.arange(600000, 600000+video_num*video_len, video_len).astype(np.int32)
# output_start = np.arange(600000+input_seq, 600000+video_num*video_len, video_len).astype(np.int32)



# print (input_start.shape,input_start)
# print (output_start.shape, output_start)
# print (np.ones(video_num).astype(np.int32).shape)
input_start = np.stack((input_start, input_seq * np.ones(video_num).astype(np.int32)))
output_start = np.stack((output_start, (video_len-input_seq) * np.ones(video_num).astype(np.int32)))
input_start = np.transpose(input_start)
output_start = np.transpose(output_start)
# print (input_start.shape,input_start)
# print (output_start.shape, output_start)
clips = np.stack((input_start, output_start))
print (clips.shape)
np.savez(op.join(arr_dir, "normalized_train_video.npz"), dims=dims, clips=clips, input_raw_data=train_videos)


train_video (600000, 3, 20, 20, 20)
[[ 3 20 20 20]]
(2, 40000, 2)


In [13]:
print ('output_start', output_start, output_start.shape)

output_start [[ 600007       8]
 [ 600022       8]
 [ 600037       8]
 ...
 [1199962       8]
 [1199977       8]
 [1199992       8]] (40000, 2)


In [5]:
test_seen_file = op.join(arr_dir, "test_seen_path.npy")
test_seen_path = np.load(test_seen_file)
print ('test_seen_path', test_seen_path.shape)

test_seen_path = np.reshape(test_seen_path, (100, -1, test_seen_path.shape[1], test_seen_path.shape[2]))
print ('test_seen_path', test_seen_path.shape)

test_seen_path (20000, 14, 3)
test_seen_path (100, 200, 14, 3)


In [6]:
test_seen_videos = generate_video(total_maz, test_seen_path, True, 0)
video_num = test_seen_videos.shape[0] * test_seen_videos.shape[1]
video_len = test_seen_videos.shape[2]
test_seen_videos = test_seen_videos/255.0
np.save(op.join(arr_dir, "test_seen_video.npy"), test_seen_videos)

videos (1, 200, 14, 3, 20, 20, 20)
videos (2, 200, 14, 3, 20, 20, 20)
videos (3, 200, 14, 3, 20, 20, 20)
videos (4, 200, 14, 3, 20, 20, 20)
videos (5, 200, 14, 3, 20, 20, 20)
videos (6, 200, 14, 3, 20, 20, 20)
videos (7, 200, 14, 3, 20, 20, 20)
videos (8, 200, 14, 3, 20, 20, 20)
videos (9, 200, 14, 3, 20, 20, 20)
videos (10, 200, 14, 3, 20, 20, 20)
videos (11, 200, 14, 3, 20, 20, 20)
videos (12, 200, 14, 3, 20, 20, 20)
videos (13, 200, 14, 3, 20, 20, 20)
videos (14, 200, 14, 3, 20, 20, 20)
videos (15, 200, 14, 3, 20, 20, 20)
videos (16, 200, 14, 3, 20, 20, 20)
videos (17, 200, 14, 3, 20, 20, 20)
videos (18, 200, 14, 3, 20, 20, 20)
videos (19, 200, 14, 3, 20, 20, 20)
videos (20, 200, 14, 3, 20, 20, 20)
videos (21, 200, 14, 3, 20, 20, 20)
videos (22, 200, 14, 3, 20, 20, 20)
videos (23, 200, 14, 3, 20, 20, 20)
videos (24, 200, 14, 3, 20, 20, 20)
videos (25, 200, 14, 3, 20, 20, 20)
videos (26, 200, 14, 3, 20, 20, 20)
videos (27, 200, 14, 3, 20, 20, 20)
videos (28, 200, 14, 3, 20, 20, 20)
v

In [7]:
test_seen_videos = np.reshape(test_seen_videos, (-1, 3, test_seen_videos.shape[-3], test_seen_videos.shape[-2], test_seen_videos.shape[-1])).astype(np.float32)

dims = np.array([3, test_seen_videos.shape[-3], test_seen_videos.shape[-2], test_seen_videos.shape[-1]])
dims = np.expand_dims(dims, axis=0)
# print (dims)
input_seq = 1
input_start = np.arange(0, video_num*video_len, video_len).astype(np.int32)
output_start = np.arange(input_seq, video_num*video_len, video_len).astype(np.int32)
# print (input_start.shape,input_start)
# print (output_start.shape, output_start)
# print (np.ones(video_num).astype(np.int32).shape)
input_start = np.stack((input_start, input_seq * np.ones(video_num).astype(np.int32)))
output_start = np.stack((output_start, (video_len-input_seq) * np.ones(video_num).astype(np.int32)))
input_start = np.transpose(input_start)
output_start = np.transpose(output_start)
# print (input_start.shape,input_start)
# print (output_start.shape, output_start)
clips = np.stack((input_start, output_start))
print (clips.shape)
np.savez(op.join(arr_dir, "normalized_test_seen_video.npz"), dims=dims, clips=clips, input_raw_data=test_seen_videos)


(2, 20000, 2)


In [8]:
test_unseen_file = op.join(arr_dir, "test_unseen_path.npy")
test_unseen_path = np.load(test_unseen_file)
print ('test_unseen_path', test_unseen_path.shape)

test_unseen_path = np.reshape(test_unseen_path, (20, -1, test_unseen_path.shape[1], test_unseen_path.shape[2]))
print ('test_unseen_path', test_unseen_path.shape)

test_unseen_path (8000, 14, 3)
test_unseen_path (20, 400, 14, 3)


In [9]:
test_unseen_videos = generate_video(total_maz, test_unseen_path, False, 0)
video_num = test_unseen_videos.shape[0] * test_unseen_videos.shape[1]
video_len = test_unseen_videos.shape[2]
test_unseen_videos = test_unseen_videos/255.0
np.save(op.join(arr_dir, "test_unseen_video.npy"), test_unseen_videos)

videos (1, 400, 14, 3, 20, 20, 20)
videos (2, 400, 14, 3, 20, 20, 20)
videos (3, 400, 14, 3, 20, 20, 20)
videos (4, 400, 14, 3, 20, 20, 20)
videos (5, 400, 14, 3, 20, 20, 20)
videos (6, 400, 14, 3, 20, 20, 20)
videos (7, 400, 14, 3, 20, 20, 20)
videos (8, 400, 14, 3, 20, 20, 20)
videos (9, 400, 14, 3, 20, 20, 20)
videos (10, 400, 14, 3, 20, 20, 20)
videos (11, 400, 14, 3, 20, 20, 20)
videos (12, 400, 14, 3, 20, 20, 20)
videos (13, 400, 14, 3, 20, 20, 20)
videos (14, 400, 14, 3, 20, 20, 20)
videos (15, 400, 14, 3, 20, 20, 20)
videos (16, 400, 14, 3, 20, 20, 20)
videos (17, 400, 14, 3, 20, 20, 20)
videos (18, 400, 14, 3, 20, 20, 20)
videos (19, 400, 14, 3, 20, 20, 20)
videos (20, 400, 14, 3, 20, 20, 20)


In [10]:
test_unseen_videos = np.reshape(test_unseen_videos, (-1, 3, test_unseen_videos.shape[-3], test_unseen_videos.shape[-2], test_unseen_videos.shape[-1])).astype(np.float32)

dims = np.array([3, test_unseen_videos.shape[-3], test_unseen_videos.shape[-2], test_unseen_videos.shape[-1]])
dims = np.expand_dims(dims, axis=0)
# print (dims)
input_seq = 1
input_start = np.arange(0, video_num*video_len, video_len).astype(np.int32)
output_start = np.arange(input_seq, video_num*video_len, video_len).astype(np.int32)
# print (input_start.shape,input_start)
# print (output_start.shape, output_start)
# print (np.ones(video_num).astype(np.int32).shape)
input_start = np.stack((input_start, input_seq * np.ones(video_num).astype(np.int32)))
output_start = np.stack((output_start, (video_len-input_seq) * np.ones(video_num).astype(np.int32)))
input_start = np.transpose(input_start)
output_start = np.transpose(output_start)
# print (input_start.shape,input_start)
# print (output_start.shape, output_start)
clips = np.stack((input_start, output_start))
print (clips.shape)
np.savez(op.join(arr_dir, "normalized_test_unseen_video.npz"), dims=dims, clips=clips, input_raw_data=test_unseen_videos)


(2, 8000, 2)


In [11]:
print (clips)

[[[     0      1]
  [    14      1]
  [    28      1]
  ...
  [111958      1]
  [111972      1]
  [111986      1]]

 [[     1     13]
  [    15     13]
  [    29     13]
  ...
  [111959     13]
  [111973     13]
  [111987     13]]]
